In [1]:
import kfp
from google.cloud import aiplatform
from typing import NamedTuple
from kfp.v2 import dsl, compiler
from kfp.v2.dsl import (Artifact, ClassificationMetrics, Input, Metrics, Output, component, Dataset)
from google.cloud import storage
from google_cloud_pipeline_components.v1.vertex_notification_email import VertexNotificationEmailOp

In [2]:
aiplatform.init(project="trim-odyssey-390415", location="us-central1")

In [3]:
model = aiplatform.Model.upload(
    display_name="linear_regression",
    artifact_uri="gs://vertex_mlops/demo/data/model",
    serving_container_image_uri='us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-3:latest')

Creating Model
Create Model backing LRO: projects/668142834453/locations/us-central1/models/8340342153959964672/operations/4476294527404998656
Model created. Resource name: projects/668142834453/locations/us-central1/models/8340342153959964672@1
To use this Model in another session:
model = aiplatform.Model('projects/668142834453/locations/us-central1/models/8340342153959964672@1')


In [4]:
models = aiplatform.Model.list()

In [5]:
print(f"Model name: {model.display_name}, Model ID: {model.resource_name}")

Model name: linear_regression, Model ID: projects/668142834453/locations/us-central1/models/8340342153959964672


In [6]:
from typing import List
from google.cloud import aiplatform

def upload_new_model_version(
    display_name: str,
    artifact_uri: str,
    serving_container_image_uri: str,
    parent_model: str,
    is_default_version: bool,
    project: str,
    location: str,
):

    aiplatform.init(project=project, location=location)

    
    model = aiplatform.Model.upload(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container_image_uri,
        parent_model=parent_model,
        is_default_version=is_default_version
    )
    
    print(f"New model version uploaded: {model.display_name}")
    model.wait()
    
    return model

# Parámetros de entrada
project = "trim-odyssey-390415"
location = "us-central1"
artifact_uri = "gs://vertex_mlops/demo/data/model"
serving_container_image_uri = 'us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-3:latest'
parent_model = "projects/668142834453/locations/us-central1/models/8340342153959964672"  
is_default_version = True


model = upload_new_model_version(
    display_name="linear_regression",
    artifact_uri=artifact_uri,
    serving_container_image_uri=serving_container_image_uri,
    parent_model=parent_model,
    is_default_version=is_default_version,
    project=project,
    location=location
)

Creating Model
Create Model backing LRO: projects/668142834453/locations/us-central1/models/8340342153959964672/operations/6813662734010286080
Model created. Resource name: projects/668142834453/locations/us-central1/models/8340342153959964672@2
To use this Model in another session:
model = aiplatform.Model('projects/668142834453/locations/us-central1/models/8340342153959964672@2')
New model version uploaded: linear_regression


In [7]:
def create_endpoint_sample(
    project: str,
    display_name: str,
    location: str,
):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint.create(
        display_name=display_name,
        project=project,
        location=location,
    )

    print(endpoint.display_name)
    print(endpoint.resource_name)
    return endpoint


# Parámetros de entrada
project = "trim-odyssey-390415"
location = "us-central1"
display_name="linear_regression"

endpoint = create_endpoint_sample(project, display_name, location)

Creating Endpoint
Create Endpoint backing LRO: projects/668142834453/locations/us-central1/endpoints/8980175457953513472/operations/3038520346366967808
Endpoint created. Resource name: projects/668142834453/locations/us-central1/endpoints/8980175457953513472
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/668142834453/locations/us-central1/endpoints/8980175457953513472')
linear_regression
projects/668142834453/locations/us-central1/endpoints/8980175457953513472


In [8]:
from google.cloud import aiplatform

def deploy_model_to_endpoint():
    aiplatform.init(project="trim-odyssey-390415", location="us-central1")
    
    endpoint = aiplatform.Endpoint('projects/668142834453/locations/us-central1/endpoints/8980175457953513472')
    model = aiplatform.Model('projects/668142834453/locations/us-central1/models/8340342153959964672')
    
    # Desplegar el modelo en el endpoint
    model.deploy(
        endpoint=endpoint,
        machine_type="n1-standard-4",
        traffic_split={"0": 100},
        min_replica_count=1,
        max_replica_count=1
    )
    
    print(f"Model deployed to endpoint: {endpoint.resource_name}")

deploy_model_to_endpoint()

Deploying model to Endpoint : projects/668142834453/locations/us-central1/endpoints/8980175457953513472
Deploy Endpoint model backing LRO: projects/668142834453/locations/us-central1/endpoints/8980175457953513472/operations/2493584791455137792
Endpoint model deployed. Resource name: projects/668142834453/locations/us-central1/endpoints/8980175457953513472
Model deployed to endpoint: projects/668142834453/locations/us-central1/endpoints/8980175457953513472


In [ ]:
from google.cloud import aiplatform_v1beta1
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

def create_batch_prediction_job_bigquery(
    project: str,
    display_name: str,
    model_name: str,
    bigquery_source_input_uri: str,
    bigquery_destination_output_uri: str,
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    
    client = aiplatform_v1beta1.JobServiceClient(client_options=client_options)
    
    model_parameters_dict = {}
    model_parameters = json_format.ParseDict(model_parameters_dict, Value())

    batch_prediction_job = {
        "display_name": display_name,
        "model": model_name,
        "model_parameters": model_parameters,
        "input_config": {
            "instances_format": "bigquery", 
            "bigquery_source": {"input_uri": bigquery_source_input_uri},
        },
        "output_config": {
            "predictions_format": "bigquery",
            "bigquery_destination": {"output_uri": bigquery_destination_output_uri},
        },
        "dedicated_resources": {
            "machine_spec": {
                "machine_type": "n1-standard-4",
            },
            "starting_replica_count": 1,
            "max_replica_count": 1,
        },
        # Model monitoring service is triggered if provided following configs.
        "model_monitoring_config": {
            "alert_config": {
                "email_alert_config": {
                    "user_emails": ["a.luisaybar@gmail.com"]
                }
            },
            "objective_configs": [
                {
                    "training_dataset": {
                        "data_format": "bigquery",
                        "bigquery_source": {"input_uri": "bq://trim-odyssey-390415.laybarm.xtrain"}
                    },
                    "training_prediction_skew_detection_config": {
                        "skew_thresholds": {
                            "MSSubClass": {"value": 0.00001},
                        }
                    }
                }
            ]
        },
    }
    
    parent = f"projects/{project}/locations/{location}"
    
    response = client.create_batch_prediction_job(
        parent=parent, batch_prediction_job=batch_prediction_job
    )
    
    print("response:", response)

# Example usage
create_batch_prediction_job_bigquery(
    project="trim-odyssey-390415",
    display_name="linear_regression",
    model_name="projects/668142834453/locations/us-central1/models/8340342153959964672",
    bigquery_source_input_uri="bq://trim-odyssey-390415.laybarm.xtest",
    bigquery_destination_output_uri="bq://trim-odyssey-390415.laybarm.predictions14"
)
